In [1]:
from lib2to3.fixes.fix_input import context
from utils import LLMModels
from langchain_core.prompts import ChatPromptTemplate


ModuleNotFoundError: No module named 'dotenv'

In [8]:
prompt_template = """
Context: {context}

Question: {question}

Instructions: Provide a clear, concise, and accurate answer to the question based on the context. Make sure to consider all relevant details from the context provided.
"""
prompt = ChatPromptTemplate.from_template(prompt_template)
prompt.format(
    context = "Marry sister is suannana",
    question= "who is Marry?"
)

'Human: \nContext: Marry sister is suannana\n\nQuestion: who is Marry?\n\nInstructions: Provide a clear, concise, and accurate answer to the question based on the context. Make sure to consider all relevant details from the context provided.\n'

In [2]:
given_models = {
    "1": "gpt-3.5-turbo",
    "2": "mixtral:8x7b",
    "3": "llama3.1:8b",
}
model_name = given_models['3']
llm_models = LLMModels(model_name=model_name)
model , embeddings = llm_models.get_llm_model()


In [10]:
sample_message =  "Hello bruh! This is Monday, March 20."
chain = prompt | model


response_message = chain.invoke({
    "context" : "Marry sister is suannana",
    "question" :"who is Marry?"
})

In [11]:
response_message

'Based on the context that "Marry sister is [something]" (and assuming it\'s a typographical error for "Mary"), I will provide an answer.\n\nIn this context, "Marry" likely means "Mary", which is a proper noun, a name. Therefore, the answer to the question "who is Marry?" would be:\n\nMary'

In [14]:
response_message.input

AttributeError: 'str' object has no attribute 'input'